### Transformação do modelo conceitual para o modelo físico

In [1]:
#Importações das bibliotecas
import pandas as pd
from connector import connection_warehouse

In [2]:
df = pd.read_csv("../data/drun_data.csv")

df.head()

,Unnamed: 0,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,tolls_amount,total_amount,payment_type,pickup_period,dropoff_period
0,0,2.0,2021-01-01 12:15:56,2021-01-01 12:19:52,1.0,43,151,1.0,1.01,5.5,0.00,0.0,6.80,2.0,AM,AM
1,1,2.0,2021-01-01 12:25:59,2021-01-01 12:34:44,1.0,166,239,1.0,2.53,10.0,2.81,0.0,16.86,1.0,AM,AM
2,2,2.0,2021-01-01 12:45:57,2021-01-01 12:51:55,1.0,41,42,1.0,1.12,6.0,1.00,0.0,8.30,1.0,AM,AM
3,3,2.0,2020-12-31 11:57:51,2021-01-01 12:04:56,1.0,168,75,1.0,1.99,8.0,0.00,0.0,9.30,2.0,PM,PM
4,4,2.0,2021-01-01 12:16:36,2021-01-01 12:16:40,2.0,265,265,3.0,0.00,-52.0,0.00,0.0,-52.80,3.0,AM,AM


In [ ]:
df = df[["VendorID", "lpep_pickup_datetime", "lpep_dropoff_datetime", "RatecodeID", "PULocationID", "DOLocationID", "passenger_count", "trip_distance", "fare_amount", "tip_amount", "tolls_amount", "total_amount", "payment_type", "pickup_period", "dropoff_period"]]

df.info()

In [110]:
# Otimização da memória

df["VendorID"] = df["VendorID"].astype("float32")
df["trip_distance"] = df["trip_distance"].astype("float32")
df["tip_amount"] = df["tip_amount"].astype("float32")
df["fare_amount"] = df["fare_amount"].astype("float32")
df["tolls_amount"] = df["tolls_amount"].astype("float32")
df["total_amount"] = df["total_amount"].astype("float32")

df["RatecodeID"] = df["RatecodeID"].astype("int32")
df["passenger_count"] = df["passenger_count"].astype("int32")
df["payment_type"] = df["payment_type"].astype("int32")

df["PULocationID"] = df["PULocationID"].astype("category")
df["DOLocationID"] = df["DOLocationID"].astype("category")

df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
df['pickup_period'] = df['pickup_period'].astype("category")
df['dropoff_period'] = df['dropoff_period'].astype("category")


In [ ]:
df.info()

In [ ]:
datetime = df[["lpep_pickup_datetime", "lpep_dropoff_datetime", "pickup_period", "dropoff_period"]]

datetime['pickup_year'] = datetime['lpep_pickup_datetime'].dt.year
datetime['pickup_month'] = datetime['lpep_pickup_datetime'].dt.month
datetime['pickup_day'] = datetime['lpep_pickup_datetime'].dt.day
datetime['pickup_hours'] = datetime['lpep_pickup_datetime'].dt.time

datetime['dropoff_year'] = datetime['lpep_dropoff_datetime'].dt.year
datetime['dropoff_month'] = datetime['lpep_dropoff_datetime'].dt.month
datetime['dropoff_day'] = datetime['lpep_dropoff_datetime'].dt.day
datetime['dropoff_hours'] = datetime['lpep_dropoff_datetime'].dt.time

datetime['pickup_hours'] = datetime['pickup_hours'].apply(lambda x: x.strftime("%H:%M:%S"))
datetime['dropoff_hours'] = datetime['dropoff_hours'].apply(lambda x: x.strftime("%H:%M:%S"))

datetime['datetime_id'] = datetime.index

In [113]:
datetime = datetime[['datetime_id', 'lpep_pickup_datetime', 'pickup_year', 'pickup_month', 'pickup_day', 'pickup_hours', 'pickup_period', 'lpep_dropoff_datetime', 'dropoff_year', 'dropoff_month', 'dropoff_day', 'dropoff_hours', 'dropoff_period']]

In [114]:
connection_warehouse(datetime, 'datetime')

Success


In [115]:
passenger = df[["passenger_count"]].drop_duplicates()
passenger = passenger.reset_index(drop=True)
passenger['passenger_id'] = passenger["passenger_count"]
passenger = passenger[['passenger_id', 'passenger_count']]
passenger = passenger[passenger['passenger_id'] <= 5]

In [116]:
connection_warehouse(passenger, 'passenger')

Success


In [117]:
ratecode_map = {
    1: "Standard rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated fare",
    6: "Group ride"
}

ratecode = df[['RatecodeID']].drop_duplicates()
ratecode = ratecode.reset_index(drop=True)
ratecode['ratecode_id'] = ratecode['RatecodeID']
ratecode['ratecode_description'] = ratecode['RatecodeID'].map(ratecode_map)
ratecode = ratecode[['ratecode_id', 'ratecode_description']]
ratecode = ratecode.drop(index=4)
ratecode = ratecode.sort_values(by= ["ratecode_id"])
ratecode


,ratecode_id,ratecode_description
0,1,Standard rate
1,2,JFK
5,3,Newark
3,4,Nassau or Westchester
2,5,Negotiated fare
6,6,Group ride


In [118]:
connection_warehouse(ratecode, 'ratecode')

Success


In [119]:
payment_map = {
    1 : "Credit card",
    2 : "Cash",
    3 : "No charge",
    4 : "Dispute",
    5 : "Unknown",
    6 : "Voided trip"
}

payment = df['payment_type'].drop_duplicates()
payment = payment.reset_index(drop=True)
payment = pd.DataFrame(payment)
payment['payment_id'] = payment['payment_type']
payment['payment_description'] = payment['payment_type'].map(payment_map)
payment = payment[['payment_id', 'payment_description']]
payment = payment.sort_values(by= ["payment_id"])
payment

,payment_id,payment_description
1,1,Credit card
0,2,Cash
2,3,No charge
3,4,Dispute
4,5,Unknown


In [120]:
connection_warehouse(payment, 'payment')

Success


In [121]:
df_zones = pd.read_csv("data/zones.csv")
df_zones = df_zones.drop("the_geom", axis=1)

In [122]:
connection_warehouse(df_zones, 'zones')

Success


In [ ]:
factory_table = df[["VendorID", "RatecodeID", "PULocationID", "DOLocationID", "passenger_count", "trip_distance", "fare_amount", "tip_amount", "tolls_amount", "total_amount", "payment_type"]]

factory_table.rename(columns={"RatecodeID" : "ratecode_id", "passenger_count" : "passenger_id", "payment_type" : "payment_id", "PULocationID" : "pickup_location_id", "DOLocationID" : "dropoff_location_id"}, inplace=True)

factory_table['datetime_id'] = datetime.loc[: , "datetime_id"]

In [ ]:
factory_table['VendorID'] = factory_table['VendorID'].astype(int)
factory_table['ratecode_id'] = factory_table['ratecode_id'].astype(int)
factory_table['passenger_id'] = factory_table['ratecode_id'].astype(int)
factory_table['payment_id'] = factory_table['payment_id'].astype(int)
factory_table['datetime_id'] = factory_table['datetime_id'].astype(int)
factory_table['pickup_location_id'] = factory_table['pickup_location_id'].astype(int)
factory_table['dropoff_location_id'] = factory_table['dropoff_location_id'].astype(int)


In [125]:
factory_table = factory_table[["VendorID", "datetime_id", "passenger_id", "trip_distance", "pickup_location_id", "dropoff_location_id", "ratecode_id", "payment_id", "fare_amount", "tip_amount", "tolls_amount", "total_amount"]]

In [126]:
connection_warehouse(factory_table, 'factory_table')

Success
